#### ############### ########### ##################### #######
#### ############### PART-II BEGIN AGAIN #####################
#### ############### ########### ##################### #######

In [1]:
## Part II: Load saved abstract features and proceed
#           with modeling and prediction

# 1.0 Call libraries
%reset -f
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import time, os

/home/ashok/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# 2. Hyperparameters/Constants
# 2.1 Dimensions of our images.
img_width, img_height = 88,70  # 150, 150
nb_train_samples = 5216
nb_validation_samples = 16
nb_test_samples= 624
epochs = 50
batch_size = 32


#2 Files for start of Using learned model

#Features Saved in files
# Bottleneck features for train data
bf_filename = '/home/ashok/.keras/models/bottleneck_features_train.npy'
# Validation-bottleneck features filename
val_filename = '/home/ashok/.keras/models/bottleneck_features_validation.npy'
# test-bottleneck features filename
test_filename = '/home/ashok/.keras/models/bottleneck_features_test.npy'


# 2.2 File to which FC model weights could be stored
top_model_weights_path = '/home/ashok/.keras/models/bottleneck_fc_model.h5'

In [3]:
# 3. Load first train features
train_data_features = np.load(open(bf_filename,'rb'))

# 3.1 Train lables. First half are of one kind and next half of other
#     Remember we had put 'shuffle = False' in data generators
#     1341 labels of one kind. Another 3875 labels of another kind
train_labels = np.array([0] * 1341 + [1] * 3875)   

# 3.2
train_data_features.shape

(5216, 2, 2, 512)

In [4]:
# 4. Validation features
validation_data_features = np.load(open(val_filename,'rb'))

# 4.2 Validation labels: half-half
validation_labels = np.array([0] * 8 + [1] * 8)

# 4.1
validation_data_features.shape  ## Is not good  for use since the features shape is not proper.

(0,)

In [5]:
#5. test features
test_data_features = np.load(open(test_filename,'rb'))

# 5.2 test labels: 
test_labels = np.array([0] * 222 + [1] * 386)

# 5.1
test_data_features.shape

(608, 2, 2, 512)

In [6]:
# 6. Plan model with FC layers only
#    We use transformed features as input to FC model instead of actual train data

model = Sequential()
model.add(Flatten(input_shape=train_data_features.shape[1:]))     # (2, 2, 512)
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# 6.1
model.compile(
              optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 524,801
Trainable params: 524,801
Non-trainable params: 0
_________________________________________________________________


In [7]:
# 6.3 Fit model and make predictions on validation dataset
#     Takes 2 minutes
#     Watch Validation loss and Validation accuracy (around 81%)
start = time.time()
model.fit(train_data_features, train_labels,
           epochs=epochs,
           batch_size=batch_size,
           validation_data=(test_data_features, test_labels),
           verbose =1
           )
end = time.time()
print("Time taken: ",(end - start)/60, "minutes")

Train on 5216 samples, validate on 608 samples
Epoch 1/50
5216/5216 [==============================] - 15s 3ms/step - loss: 0.2294 - acc: 0.9062 - val_loss: 0.8575 - val_acc: 0.7484
Epoch 2/50
5216/5216 [==============================] - 11s 2ms/step - loss: 0.1484 - acc: 0.9415 - val_loss: 0.4676 - val_acc: 0.8355
Epoch 3/50
5216/5216 [==============================] - 11s 2ms/step - loss: 0.1317 - acc: 0.9467 - val_loss: 1.1326 - val_acc: 0.7352
Epoch 4/50
5216/5216 [==============================] - 14s 3ms/step - loss: 0.1156 - acc: 0.9555 - val_loss: 1.0388 - val_acc: 0.7829
Epoch 5/50
5216/5216 [==============================] - 10s 2ms/step - loss: 0.1192 - acc: 0.9565 - val_loss: 0.9555 - val_acc: 0.7993
Epoch 6/50
5216/5216 [==============================] - 11s 2ms/step - loss: 0.1095 - acc: 0.9571 - val_loss: 0.6561 - val_acc: 0.8339
Epoch 7/50
5216/5216 [==============================] - 13s 3ms/step - loss: 0.1089 - acc: 0.9592 - val_loss: 1.6658 - val_acc: 0.7122
Epoch 8/

In [8]:
# 7. Finally save model weights for later use
model.save_weights(top_model_weights_path)
#######################################################

### The mean Validation Accuracy received by this model on Test data is 80 %.

###### We have removed the Validation Model fitting since there are not a big number of validation features.